### Five qubit MUBs

Using the desarguesian spread and Kantors inequivalent set.

In [1]:
p = 2
N = 5

F = GF(p^N, 'x', repr='poly')
x = F.gen()

In [2]:
charpoly(x) # defaults to Conway polynomial

x^5 + x^2 + 1

In [3]:
# GR(4,5)
R = PolynomialRing(Integers(4), 't')
t = R.gen()

In [4]:
factor(t^5 + t^2 + 1)

t^5 + t^2 + 1

In [5]:
GR = R.quotient(t^5 + t^2 + 1, 'w')
w = GR.gen()
GR

Univariate Quotient Polynomial Ring in w over Ring of integers modulo 4 with modulus t^5 + t^2 + 1

Field generator is not the Teichmüller generator, this is one is though:

In [7]:
xi = w^2 + 2*w + 1
xi^(2^5-1)

1

In [33]:
T = [GR(0)] + [xi^j for j in range(1, 2^N-1)] + [GR(1)]

In [24]:
def Proj(u, v=None):
    if not v:
        v = u
    u = matrix(u).transpose()
    v = matrix(v).transpose()
    return u.tensor_product(v.conjugate_transpose())

Id = identity_matrix(SR, 2^N)

def toInt(k):
    return list(F).index(k)

def chi(k):
    return exp(2 * pi * I * int(k.trace()) / 2)

In [34]:
def TeichLift(k):
    return T[toInt(k)]
hat = TeichLift

In [35]:
for a in F:
    for b in F:
        if hat(a * b) != hat(a) * hat(b):
            raise Exception('Not multiplicative group homomorphism!')

In [36]:
def Fourier():
    s = zero_matrix(SR, 2^N, 2^N)
    for i, a in enumerate(F):
        for j, b in enumerate(F):
            s[i,j] = chi(a * b) / sqrt(2^N)
    return s
FF = Fourier()

def Z(a):
    return diagonal_matrix([chi(a * k) for k in F])

def X(b):
    return FF.conjugate_transpose() * Z(b) * FF

In [28]:
def phi(tau, nu):
    return (I)^(int((tau * nu).trace()))

def D(a, b):
    return phi(hat(a), hat(b)) * Z(a) * X(b)

def V(mu):
    s = zero_matrix(SR, 2^N, 2^N)
    for i, k in enumerate(F):
        s += phi(hat(k), hat(k) * hat(mu)) * Proj(vector(FF[:,i]))
    return s

In [39]:
# Kantor's MUBs
def VK(mu):
    s = zero_matrix(SR, 2^N, 2^N)
    m = hat(mu)
    curve = lambda k: m^2 * k + m * k.trace() + (m * k).trace()
    for i, k in enumerate(F):
        s += phi(hat(k), curve(hat(k))) * Proj(vector(FF[:,i]))
    return s

In [29]:
# Orthonormality
def isOrthonormal(m):
    return m.conjugate_transpose() * m == Id

# MUBs
def isMUB(m1, m2):
    m = (
        m1.conjugate_transpose() * m2
    ).apply_map(lambda t: abs(t)^2)
    return m == ones_matrix(2^N, 2^N) / (2^N)

def checkMUBs(mubs):
    if type(mubs) != list:
        mubs_list = []
        for i in range(2^N+1):
            mubs_list.append(mubs[(i*2^N):(i+1)*2^N,:])
    else:
        mubs_list = mubs
            
    for i in range(2^N+1):
        for j in range(2^N+1):
            if i == j:
                if not isOrthonormal(mubs_list[i]):
                    raise Exception(
                        'Encountered a basis that is not orthonormal!',
                        i
                    )
            else:
                if not isMUB(mubs_list[i], mubs_list[j]):
                    raise Exception(
                        'Encountered non-MUB pairs of basis!',
                        i, j
                    )
    return True

In [50]:
mubsStd = [FF] + [VK(mu) for mu in F]

In [51]:
checkMUBs(mubsStd)

Exception: ('Encountered non-MUB pairs of basis!', 1, 2)